In [1]:
from sklearn import svm
import spacy
from string import punctuation

<b> The following code requires spacy to be installed. It also requires the download of its english datasets, available on the spacy website. Please replace the path string below with the absolute path of the downloaded english dataset of spacy</b>

In [3]:
nlp = spacy.load("/usr/local/lib/python2.7/dist-packages/spacy/data/en/en_core_web_sm-2.0.0")

Please replace 'training.data' in the open() line with the relative path of the training file.

In [5]:
allWords=dict()   
sentences=[]
label1=[]
label2=[]
lab1Count=1
lab2Count=1
label1dict=dict()
label2dict=dict()

for line in open('training.data','r',encoding="ISO-8859-1"):
    line=[word.lower() for word in line.strip().split(' ')]
    labels=line[0].split(':')
    if(len(labels)!=2 or ' ' in line[0]):
        print('Formatting asssumption wrong')
        break
    sentence=[word for word in line[1:] if word not in punctuation]
    sentences.append(sentence)
    if(not(label1dict.get(labels[0],0))):
        label1dict[labels[0]]=lab1Count
        lab1Count+=1
    if(not(label2dict.get(labels[1],0))):
        label2dict[labels[1]]=lab2Count
        lab2Count+=1
    label1.append(label1dict[labels[0]])
    label2.append(label2dict[labels[1]])


The basic principle is: The algorithm only depends on the properties of the question word and its neighbour. 
In each sentence, I look for the question word using spacy. Then I represent each sentence using a vector that has the structure: [POS of the Q-word, the Q-word, POS of the next word, the next word]. I have never tried this approach before and have borrowed the idea from  https://nlp.stanford.edu/courses/cs224n/2010/reports/olalerew.pdf. However I have ignored several aspects of the methodology. 


In [8]:
sentencesStr=[" ".join(sentences[i]) for i in range(len(sentences))]

In [9]:
qTag=dict()
qTagInd=1
qText=dict()
qTextInd=1
qBigram=dict()
qBigramInd=1
qNbrPos=dict()
qNbrInd=1
rootTag=dict()
rootTagInd=1
allVectors=[]
count=1
allBigrams=[]

for sentence in sentencesStr:
    sentenceVector=[None]*4
    tokens=nlp(sentence)
    for token in tokens:
        
        if token.tag_ == "WDT" or token.tag_ == "WP" or token.tag_ == "WP$" or token.tag_ == "WRB":
            if(token.tag_ not in qTag):
                qTag[token.tag_]=qTagInd
                qTagInd+=1
            sentenceVector[0]=qTag[token.tag_]
            
            if(token.text not in qText):
                qText[token.text]=qTextInd
                qTextInd+=1
            sentenceVector[1]=qText[token.text]
                
            if(token.i<len(tokens)-1):
                biGram=(str(tokens[token.i]),str(tokens[token.i+1]))
                if biGram not in qBigram:
                    allBigrams.append(biGram)
                    qBigram[biGram]=qBigramInd
                    qBigramInd+=1
                sentenceVector[2]=qBigram[biGram]
                    
                if(tokens[token.i+1].tag_ not in qNbrPos):
                    qNbrPos[tokens[token.i+1].tag_]=qNbrInd
                    qNbrInd+=1
                sentenceVector[2]=qNbrPos[tokens[token.i+1].tag_]
                    
        if token.dep_ == "ROOT":
            if token.tag_ not in rootTag:
                rootTag[token.tag_]=rootTagInd
                rootTagInd+=1
            sentenceVector[3]=rootTag[token.tag_]
    sentenceVector=[0 if word==None else word for word in sentenceVector]
    allVectors.append(sentenceVector)

The below function converts each vector described above into a 66 dimensional vector.

In [12]:
def makeVectors(vec):
    rep=[0]*(62+4)
    rep[vec[0]]=1
    rep[5+vec[1]]=1
    rep[20+vec[2]]=1
    rep[44+vec[3]]=1
    return rep
    
    
    

In [14]:
vectors=[]
for vec in allVectors:
    vectors.append(makeVectors(vec))
    

In [19]:
import numpy as np

In [20]:
#Training set
trainVectors=np.array(vectors[:int(0.6*len(vectors))])
trainLabels1=np.array(label1[:int(0.6*len(vectors))])
trainLabels2=np.array(label2[:int(0.6*len(vectors))])

In [21]:
#Validation Set
validVectors=np.array(vectors[int(0.6*len(vectors)):int(0.8*len(vectors))])
validLabels1=np.array(label1[int(0.6*len(vectors)):int(0.8*len(vectors))])
validLabels2=np.array(label2[int(0.6*len(vectors)):int(0.8*len(vectors))])

In [22]:
#test Set
testVectors=np.array(vectors[int(0.8*len(vectors)):])
testLabels1=np.array(label1[int(0.8*len(vectors)):])
testLabels2=np.array(label2[int(0.8*len(vectors)):])

In [23]:
for c in [100,1000,10**5]:
    clf=svm.SVC(C=c, kernel='rbf', gamma='auto', tol=0.001, class_weight='balanced', max_iter=-1,\
                decision_function_shape='ovr')
    clf.fit(trainVectors,trainLabels1)
    preds=clf.predict(validVectors)
    accu=sum([1 if i==j else 0 for i,j in zip(preds,validLabels1)])/len(validLabels1)
    print(accu)

0.610032362459547
0.6203883495145631
0.6229773462783171


In [24]:
clf=svm.SVC(C=10**5, kernel='rbf', gamma='auto', tol=0.001, class_weight='balanced', max_iter=-1,\
                decision_function_shape='ovr')
clf.fit(np.vstack([trainVectors,validVectors]),np.hstack([trainLabels1,validLabels1]))
preds=clf.predict(testVectors)
accu=sum([1 if i==j else 0 for i,j in zip(preds,testLabels1)])/len(testLabels1)
print('Test accuracy obtained for primary label is ',accu)

Test accuracy obtained for primary label is  0.620834681332902


In [25]:
for c in [100,1000,10**5]:
    clf=svm.SVC(C=c, kernel='rbf', gamma='auto', tol=0.001, class_weight='balanced', max_iter=-1,\
                decision_function_shape='ovr')
    clf.fit(trainVectors,trainLabels2)
    preds=clf.predict(validVectors)
    accu=sum([1 if i==j else 0 for i,j in zip(preds,validLabels2)])/len(validLabels2)
    print(accu)

0.33980582524271846
0.3423948220064725
0.34368932038834954


In [26]:
clf=svm.SVC(C=10**5, kernel='rbf', gamma='auto', tol=0.001, class_weight='balanced', max_iter=-1,\
                decision_function_shape='ovr')
clf.fit(np.vstack([trainVectors,validVectors]),np.hstack([trainLabels2,validLabels2]))
preds=clf.predict(testVectors)
accu=sum([1 if i==j else 0 for i,j in zip(preds,testLabels2)])/len(testLabels2)
print("Test accuracy obtained for secondary label is",accu)

Test accuracy obtained for secondary label is 0.33904885150436753


Clearly, the representation that I tried was inadequate. I am unable to improve the performance of the model by including a few other meta parameters of the sentence. I have not included the code of the same, in the interest of code cleanliness.